# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [6]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [7]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [9]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [10]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster 1    46
Cluster 5    32
Cluster 2    29
Cluster 4    29
Cluster 0    27
Cluster 3    27
Cluster 7    20
Cluster 6    19
Name: Cluster, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [11]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [16]:
print(cluster_descriptions["Cluster 5"])

age - 45-54: 28, 25-34: 3, unknown: 1, 18-24: 0, 35-44: 0, 55-64: 0, <18: 0, >=65: 0
edu_level - Wyższe: 28, Średnie: 4, Podstawowe: 0
fav_animals - Psy: 23, Koty: 5, Koty i Psy: 3, Inne: 1, Brak ulubionych: 0
fav_place - Nad wodą: 22, Inne: 2, W górach: 0, W lesie: 0
gender - Mężczyzna: 17, Kobieta: 15



In [24]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 7:
age - 35-44: 12, 25-34: 3, 45-54: 1, 55-64: 1, <18: 1, >=65: 1, unknown: 1, 18-24: 0
edu_level - Wyższe: 17, Średnie: 2, Podstawowe: 1
fav_animals - Brak ulubionych: 12, Koty: 8, Inne: 0, Koty i Psy: 0, Psy: 0
fav_place - Nad wodą: 10, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 15, Kobieta: 4


Klaster Cluster 2:
age - 25-34: 10, 18-24: 8, 35-44: 5, 45-54: 5, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 29, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 12, Koty: 8, Inne: 6, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, Nad wodą: 5, W lesie: 3, Inne: 1
gender - Mężczyzna: 27, Kobieta: 2


Klaster Cluster 0:
age - 45-54: 10, 35-44: 8, 25-34: 5, 55-64: 4, 18-24: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 25, Średnie: 2, Podstawowe: 0
fav_animals - Psy: 25, Brak ulubionych: 1, Koty: 1, Inne: 0, Koty i Psy: 0
fav_place - W lesie: 27, Inne: 0, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 20, Kobi

In [25]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [26]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [27]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [28]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Lasu i Psów', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu w lesie i mają wyższe wykształcenie. Większość z nich to mężczyźni w wieku 45-54 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Wodni Entuzjaści z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu nad wodą i mają wyższe wykształcenie. Większość z nich to mężczyźni w wieku 35-44 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Górscy Miłośnicy ze Średnim Wykształceniem', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu w górach i mają średnie wykształcenie. Większość z nich to mężczyźni w wieku 25-34 lat, a ich ulubionymi zwierzętami są psy i koty.'}, 'Cluster 3': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu w górach i mają wyższe wykształcenie. Większość z